In [1]:
import numpy as np

from agent import *
from environment import *
from behavior import *
from controller import Controller

Using TensorFlow backend.


In [2]:
def getEnvironmentTraining():
    return MTurkSurveyUser(filePaths=[
            'survey/ver2_mturk/results/01_1st_Batch_3137574_batch_results.csv',
            'survey/ver2_mturk/results/02_Batch_3148398_batch_results.csv',
            'survey/ver2_mturk/results/03_Batch_3149214_batch_results.csv',
    ], filterFunc=(lambda r: ord(r['rawWorkerID'][-1]) % 2 == 0), dismissWarningMsg=True)

def getEnvironmentTesting():
    return MTurkSurveyUser(filePaths=[
            'survey/ver2_mturk/results/01_1st_Batch_3137574_batch_results.csv',
            'survey/ver2_mturk/results/02_Batch_3148398_batch_results.csv',
            'survey/ver2_mturk/results/03_Batch_3149214_batch_results.csv',
    ], filterFunc=(lambda r: ord(r['rawWorkerID'][-1]) % 2 == 1), dismissWarningMsg=True)

In [3]:
def getResponseRates(results):
    """
    Returns: (# notifications, response rate)
    """
    notificationEvents = [r for r in results if r['decision']]
    numNotifications = len(notificationEvents)
    numAcceptedNotifications = len([r for r in notificationEvents if r['reward'] > 0])
    numDismissedNotifications = len([r for r in notificationEvents if r['reward'] < 0])
    numAcceptsDismisses = numAcceptedNotifications + numDismissedNotifications
    responseRate = numAcceptedNotifications / numAcceptsDismisses if numAcceptsDismisses > 0 else 0
    return (numNotifications, responseRate)

In [8]:
tmpModelPath = '/tmp/model.txt'
numRounds = 1

In [11]:
agents = [SVMAgent, NNAgent]
trainingSource = [
    (getEnvironmentTesting, 2),
    (getEnvironmentTesting, 4),
    (getEnvironmentTesting, 10),
]
behaviorSource = [
    'behavior/data/2.txt',
    'behavior/data/4.txt',
    'behavior/data/5.txt',
    'behavior/data/6.txt',
]

numAgents = len(agents)
numTrainingTasks = len(trainingSource)
resultResponseRatio = np.zeros((numAgents, numTrainingTasks))

In [12]:
taskResults = []
for iAgent, AgentClass in enumerate(agents):
    for iTraining, trainingParams in enumerate(trainingSource):
        for iBehavior, behaviorFile in enumerate(behaviorSource):
        
            taskNumNotificationHolder = np.zeros(numRounds)
            taskResponseRateHolder = np.zeros(numRounds)

            for iRound in range(numRounds):
                trainingEnvorinmentFunc, numTrainingWeeks = trainingParams

                # collecting training data
                agent = ClassifierDataCollectionAgent()
                environment = trainingEnvorinmentFunc()
                behavior = ExtraSensoryBehavior(behaviorFile)
                controller = Controller(agent, environment, behavior, simulationWeek=numTrainingWeeks,
                                        negativeReward=-5, verbose=False)
                controller.execute()
                agent.saveModel(tmpModelPath)

                # testing phase
                agent = AgentClass()
                agent.loadModel(tmpModelPath)
                environment = getEnvironmentTesting()
                controller = Controller(agent, environment, behavior, simulationWeek=10,
                                        negativeReward=-5, verbose=False)
                results = controller.execute()
                numNotifications, responseRate = getResponseRates(results)

                taskNumNotificationHolder[iRound] = numNotifications
                taskResponseRateHolder[iRound] = responseRate

            taskResults.append((AgentClass, trainingParams, behaviorFile,
                  np.mean(taskNumNotificationHolder), np.mean(taskResponseRateHolder)))

for taskResult in taskResults:
    print(taskResult)

Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish training the model
Begin to train the model
Finish

In [29]:
print(resultReward)
print(resultResponseRatio)

[[ -954.6  -243.1  -466.3  -234.9  -377.1  -158.1  -189.2  -265.4  -210.2
   -127.5   -49.5]
 [-1039.4  -141.4  -129.   -162.    -35.5   -62.5  -134.7   -34.6  -130.7
    -32.1   -19. ]]
[[0.720011   0.53228435 0.59490083 0.79537774 0.76666028 0.63824792
  0.80389902 0.62358735 0.81888067 0.66287428 0.83295978]
 [0.69154703 0.29472797 0.23193012 0.3896916  0.67577955 0.57304612
  0.4013819  0.76618782 0.6599013  0.73799442 0.52544098]]
